In [1]:
import sys
sys.path.append("..")

from glob import glob
import matplotlib.pyplot as plt
import ipywidgets as ipw
from IPython.display import Audio
import numpy as np
import pickle
import yaml
import pandas as pd

from lib.notebooks import plot_groups_metrics
from imitative_agent import ImitativeAgent

In [2]:
agents_path = glob("../out/imitative_agent/*/")
agents_path.sort()

In [7]:
groups_metrics = {}
agents_loss = {
    "path": [],
    "datasets": [],
    "inverse_learning_rate": [],
    "inverse_layers": [],
    "inverse_dropout_p": [],
    "direct_learning_rate": [],
    "direct_layers": [],
    "direct_dropout_p": [],
    "jerk_weight": [],
    "direct_estimation_error": [],
    "inverse_estimation_error": [],
    "jerk": [],
    "repetition_error": [],
}

for agent_path in agents_path:
    agent = ImitativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    agents_loss["path"].append(agent_path[-5:-1])
    agents_loss["datasets"].append(",".join(agent.config['dataset']['names']))

    training_config = config["training"]
    inverse_model_learning_rate = training_config["inverse_model_learning_rate"] if "inverse_model_learning_rate" in training_config else training_config["learning_rate"]
    direct_model_learning_rate = training_config["direct_model_learning_rate"] if "direct_model_learning_rate" in training_config else training_config["learning_rate"]

    agents_loss["inverse_learning_rate"].append(inverse_model_learning_rate)
    agents_loss["inverse_layers"].append(f"{config['model']['inverse_model']['num_layers']}x{config['model']['inverse_model']['hidden_size']}")
    agents_loss["inverse_dropout_p"].append(config['model']['inverse_model']['dropout_p'])

    if 'use_synth_as_direct_model' not in config['model']:
        agents_loss["direct_learning_rate"].append(direct_model_learning_rate)
        agents_loss["direct_layers"].append(f"{len(config['model']['direct_model']['hidden_layers'])}x{config['model']['direct_model']['hidden_layers'][0]}")
        agents_loss["direct_dropout_p"].append(config['model']['direct_model']['dropout_p'])
    else:
        agents_loss["direct_learning_rate"].append(0)
        agents_loss["direct_layers"].append("synth")
        agents_loss["direct_dropout_p"].append(0)
    
    agents_loss["jerk_weight"].append(config['training']['jerk_loss_weight'])

    final_loss_index = np.argmin(metrics["validation"]["inverse_model_repetition_error"])
    
    
    if 'use_synth_as_direct_model' not in config['model']:
        agents_loss["direct_estimation_error"].append(metrics["test"]["direct_model_estimation_error"][final_loss_index])
    else:
        agents_loss["direct_estimation_error"].append(0)

    agents_loss["inverse_estimation_error"].append(metrics["test"]["inverse_model_estimation_error"][final_loss_index])
    agents_loss["jerk"].append(metrics["test"]["inverse_model_jerk"][final_loss_index])
    agents_loss["repetition_error"].append(metrics["test"]["inverse_model_repetition_error"][final_loss_index])
    
    group_name = "\n".join((
        f"datasets={','.join(agent.config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
        # f"frame_padding={config['model']['sound_quantizer']['frame_padding']}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

agents_loss = pd.DataFrame(agents_loss)

/home/georges1/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.37691562762983044 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/georges1/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.011016616762974218 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/georges1/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.6155439280874151 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/georges1/anaconda3/lib/python3.7/site-packages/torch/nn

In [8]:
datasets = pd.unique(agents_loss["datasets"])
jerk_weights = pd.unique(agents_loss["jerk_weight"])

def show_top_agents(measure="repetition_error", datasets=datasets[0], jerk_weight=jerk_weights[0], use_synth_as_direct=False, ascending=True):
    subset = agents_loss[agents_loss["datasets"] == datasets]
    subset = subset[subset["jerk_weight"] == jerk_weight]
    if use_synth_as_direct:
        subset = subset[subset["direct_layers"] == "synth"]
    else:
        subset = subset[subset["direct_layers"] != "synth"]
    display(subset.sort_values(measure, ascending=ascending).head(30))

ipw.interactive(show_top_agents, measure=agents_loss, datasets=datasets, jerk_weight=jerk_weights, use_synth_as_direct=False, ascending=True)

interactive(children=(Dropdown(description='measure', index=12, options=('path', 'datasets', 'inverse_learning…

In [5]:
groups_metrics = {}

for agent_path in agents_path:
    agent = ImitativeAgent.reload(agent_path, load_nn=False)
    config = agent.config
    with open("%s/metrics.pickle" % agent_path, "rb") as f:
        metrics = pickle.load(f)
    
    if config['training']['jerk_loss_weight'] != 1:
        continue
    
    group_name = "\n".join((
        f"{','.join(config['dataset']['names'])}",
        f"synth_art={agent.synthesizer.config['dataset']['art_type']}",
        f"jerk_c={config['training']['jerk_loss_ceil']}",
        f"jerk_w={config['training']['jerk_loss_weight']}",
        f"bi={config['model']['inverse_model']['bidirectional']}",
    ))
    
    if group_name not in groups_metrics:
        groups_metrics[group_name] = {}
    groups_metrics[group_name][agent_path] = metrics

In [6]:
metrics_name = [
    "direct_model_estimation_error",
    "inverse_model_estimation_error",
    "inverse_model_repetition_error",
    "inverse_model_jerk",
]

def show_metrics(split_name="test"):
    plot_groups_metrics(groups_metrics, metrics_name, split_name)
display(ipw.interactive(show_metrics, split_name=["train", "validation", "test"]))

interactive(children=(Dropdown(description='split_name', index=2, options=('train', 'validation', 'test'), val…